In [ ]:
# Import python packages
import json
import pandas as pd
from snowflake.snowpark import functions as F
from datetime import date, timedelta
from snowflake.snowpark import types as T
from snowflake.ml.model import custom_model
from snowflake.ml.model import model_signature
from snowflake.ml.registry import registry
import numpy as np
import ast
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")

def fix_values(column):
    return F.upper(F.regexp_replace(F.col(column), "[^a-zA-Z0-9]+", "_"))

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Shows how to create a pandas dataframe
# Writes a pandas dataframe to Snowflake as a table
tspd = pd.read_csv('data/ts_data.csv')

ts = session.create_dataframe(tspd)
ts.write.mode("overwrite").save_as_table("traffic_data")
ts.show()

In [ ]:
df = session.table("traffic_data")
for col in ["HOLIDAY_NAME"]:
    df = df.with_column(col, fix_values(col))

train_start_date = '2021-01-01'
train_end_date = '2024-10-08'
test_start_date = '2024-10-22'
test_end_date = '2024-12-05'

max_forecast_str = '2024-12-19'

print("Train Start Date: 2021-01-01")
print(f"Train End Date: {train_end_date}")
print(f"Test Start Date: {test_start_date}")
print(f"Test End Date: {test_end_date}")
print(f"Forecast End Date: {max_forecast_str}")

In [ ]:
# We will be one-hot encoding Holidays
# For Case/space sensitivity we will fix all holidays

for col in ["HOLIDAY_NAME"]:
    df = df.with_column(col, fix_values(col))

df.show()

Run One Model

In [ ]:
df_features = session.table('traffic_data').filter(F.col('STORE_ID') == 1).to_pandas()
df_features['DATE'] = pd.to_datetime(df_features['DATE'])
df_features = df_features.sort_values(by='DATE', ascending=False)
df_features

For partitioned modeling I recommend getting one model to run correctly before training every partition that way you can take the code that works for one model and insert it into the partitioned modeling API

In [ ]:
import xgboost

df = df_features
# Set the date column as our index.
df['DATETIME'] = pd.to_datetime(df['DATE'])

# Set the index to the datetime column
df.set_index('DATETIME', inplace=True)
df = df.drop(columns = ['DATE'])

for column in df.columns:
    if column not in ['TRAFFIC','HOLIDAY_NAME']:
        df[column] = df[column].astype('int')
        
# Use get_dummies (one-hot encoding) for categorical features.
df = pd.get_dummies(data=df, columns=['HOLIDAY_NAME'])


train = df[(df.index >= pd.to_datetime(train_start_date)) & (df.index <= pd.to_datetime(train_end_date))]

forecast = df[(df.index >= pd.to_datetime(test_start_date)) & (df.index <= pd.to_datetime(max_forecast_str))]

# Remove the target from the input dataset, and construct target dataset.
X_train = train.drop('TRAFFIC', axis=1)
y_train = train['TRAFFIC']

X_forecast = forecast.drop('TRAFFIC', axis=1)

# Train an XGBoost regression model.
model = xgboost.XGBRegressor(n_estimators=200, n_jobs=1)
model.fit(X_train, y_train, verbose=False)

# Predict the hourly forecast for the future dates and make sure no predictions are less than zero.
forecast.loc[:, 'PREDICTION'] = model.predict(X_forecast)
forecast['DATETIME'] = forecast.index
forecast = forecast[['DATETIME', 'PREDICTION']]
forecast = forecast.sort_index()
forecast.loc[forecast['PREDICTION'] < 0, 'PREDICTION'] = 0
forecast

In [ ]:
forecast.rename(columns={'DATETIME': 'DATE'}, inplace=True)
pred_vs_actual = pd.merge(df_features[['DATE', 'TRAFFIC']], forecast[['DATE', 'PREDICTION']], on='DATE', how='inner')
pred_vs_actual

In [ ]:
import altair as alt
import streamlit as st
df = pred_vs_actual
# Replace 0's in 'TRAFFIC' after '2024-11-02' with NaN
df.loc[(df['DATE'] > '2024-11-02') & (df['TRAFFIC'] == 0), 'TRAFFIC'] = np.nan

# Set DATE as the index for better plotting
df.set_index('DATE', inplace=True)

# Create the line chart with altair
chart = alt.Chart(df.reset_index()).transform_fold(
    ['TRAFFIC', 'PREDICTION'],
    as_=['variable', 'value']
).mark_line().encode(
    x='DATE:T',
    y=alt.Y('value:Q').scale(zero=False),  # Set the y-axis starting from 500
    color='variable:N'
).properties(
    title='TRAFFIC and PREDICTION Line Chart'
)

# Display the chart in Streamlit
st.altair_chart(chart, use_container_width=True)

Run 500 models with the partitioned modeling API

In [ ]:
from snowflake.ml.model import custom_model
class ForecastingModel(custom_model.CustomModel):

    # Use the same decorator as for methods with FUNCTION inference.
    @custom_model.partitioned_inference_api
    def predict(self, df: pd.DataFrame) -> pd.DataFrame:        
        import xgboost
        
        # Set the date column as our index.
        df['DATETIME'] = pd.to_datetime(df['DATE'])
        
        # Set the index to the datetime column
        df.set_index('DATETIME', inplace=True)
        df = df.drop(columns = ['DATE'])
        
        for column in df.columns:
            if column not in ['TRAFFIC','HOLIDAY_NAME']:
                df[column] = df[column].astype('int')
                
        # Use get_dummies (one-hot encoding) for categorical features.
        df = pd.get_dummies(data=df, columns=['HOLIDAY_NAME'])
        
        
        train = df[(df.index >= pd.to_datetime(train_start_date)) & (df.index <= pd.to_datetime(train_end_date))]
        
        forecast = df[(df.index >= pd.to_datetime(test_start_date)) & (df.index <= pd.to_datetime(max_forecast_str))]
        
        # Remove the target from the input dataset, and construct target dataset.
        X_train = train.drop('TRAFFIC', axis=1)
        y_train = train['TRAFFIC']
        
        X_forecast = forecast.drop('TRAFFIC', axis=1)
        
        # Train an XGBoost regression model.
        model = xgboost.XGBRegressor(n_estimators=200, n_jobs=1)
        model.fit(X_train, y_train, verbose=False)
        
        # Predict the hourly forecast for the future dates and make sure no predictions are less than zero.
        forecast.loc[:, 'PREDICTION'] = model.predict(X_forecast)
        forecast['DATETIME'] = forecast.index
        forecast = forecast[['DATETIME', 'PREDICTION']]
        forecast = forecast.sort_index()
        forecast.loc[forecast['PREDICTION'] < 0, 'PREDICTION'] = 0

        return forecast

my_forecasting_model = ForecastingModel()

In [ ]:
# This is just a nice little fucntion to automatically
# Make versions iterate V_1, V_2

def get_next_version(reg, model_name) -> str:
    """
    Returns the next version of a model based on the existing versions in the registry.

    Args:
        reg: The registry object that provides access to the models.
        model_name: The name of the model.

    Returns:
        str: The next version of the model in the format "V_<version_number>".

    Raises:
        ValueError: If the version list for the model is empty or if the version format is invalid.
    """
    models = reg.show_models()
    if models.empty:
        return "V_1"
    elif model_name not in models["name"].to_list():
        return "V_1"
    max_version_number = max(
        [
            int(version.split("_")[-1])
            for version in ast.literal_eval(
                models.loc[models["name"] == model_name, "versions"].values[0]
            )
        ]
    )
    return f"V_{max_version_number + 1}"

In [ ]:
reg = registry.Registry(session=session)
model_name = "XGB_TRAFFIC"
options = {
    "function_type": "TABLE_FUNCTION",
}

mv = reg.log_model(
    my_forecasting_model,
    model_name=model_name,
    version_name=get_next_version(reg,model_name),
    conda_dependencies=["pandas", "scikit-learn", "xgboost"],
    options=options,
    signatures={
        "predict": model_signature.ModelSignature(
            inputs=[
                model_signature.FeatureSpec(name="DATE", dtype=model_signature.DataType.TIMESTAMP_NTZ),
                model_signature.FeatureSpec(name="STORE_ID", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="WEEK_DAY_NBR", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="MTH_DAY_NBR", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="CALENDAR_MTH", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="CALENDAR_YEAR", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="HOLIDAY_NAME", dtype=model_signature.DataType.STRING),
                model_signature.FeatureSpec(name="TRAFFIC", dtype=model_signature.DataType.DOUBLE),
            ],
            outputs=[
                model_signature.FeatureSpec(name="DATETIME", dtype=model_signature.DataType.TIMESTAMP_NTZ),
                model_signature.FeatureSpec(name="PREDICTION", dtype=model_signature.DataType.DOUBLE),
         ],
        )
    },
)

In [ ]:
df = session.table("traffic_data")
for col in ["HOLIDAY_NAME"]:
    df = df.with_column(col, fix_values(col))

df.show()

In [ ]:
df = session.table("traffic_data").with_column("DATE", F.to_timestamp(F.col("DATE"))).cache_result()
for col in ["HOLIDAY_NAME"]:
    df = df.with_column(col, fix_values(col))

results = mv.run(df, partition_column="STORE_ID")

In [ ]:
df = session.table("traffic_data").with_column("DATE", F.to_timestamp(F.col("DATE")))
for col in ["HOLIDAY_NAME"]:
    df = df.with_column(col, fix_values(col))

results = mv.run(df, partition_column="STORE_ID").select(
    F.to_date("DATETIME").alias("DATE"),"STORE_ID","PREDICTION"
)


results.show()

In [ ]:
final_XGB = results.join(
    df, ['DATE','STORE_ID']
).select(
    F.to_date(df.DATE).alias("DATE"),
    df.STORE_ID.alias("STORE_ID"),
    df.TRAFFIC,
    results.PREDICTION,
).with_column('MODEL',F.lit('XGBOOST'))
final_XGB.show()

In [ ]:
import json
from snowflake.snowpark import functions as F

def calculate_mape(df):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between actual and predicted values for each country and channel using Snowpark.

    Parameters:
    df (DataFrame): Snowpark DataFrame containing actual and predicted values.

    Returns:
    DataFrame: DataFrame containing the MAPE value as a percentage for each country and channel.
    """
    # Create a temporary column for the absolute percentage error
    mape_df = df.with_column(
        "APE",
        F.div0(F.abs(F.col('TRAFFIC') - F.col('PREDICTION')), F.col('TRAFFIC'))
    )

    # Group by COUNTRY and CHANNEL and calculate the mean of the absolute percentage error
    mape_per_store_id = mape_df.group_by(["STORE_ID"]).agg(
        (F.mean("APE") * 100).alias("MAPE")
    )

    return mape_per_store_id

In [ ]:
mape_values = calculate_mape(
    final_XGB.filter(
        (F.col("DATE") >= test_start_date) & (F.col("DATE") <= test_end_date) & (F.col("STORE_ID") <= 10)
    )
)
mape_results = mape_values.collect()

# Create a nested dictionary with COUNTRY as the outer key and CHANNEL as the inner key
mape_dict = {}
for row in mape_results:
    store_id = row[0]
    mape_value = round(row[1], 1)
    if store_id not in mape_dict:
        mape_dict[store_id] = {}
    mape_dict[store_id] = f"{mape_value}%"

mape_json = json.dumps(mape_dict, indent=4)
print(mape_json)

In [ ]:
mv.set_metric(metric_name="MAPES", value=mape_json)

In [ ]:
reg.show_models()

In [ ]:
from snowflake.ml.model import custom_model
class ForecastingModel_prophet(custom_model.CustomModel):

    # Use the same decorator as for methods with FUNCTION inference.
    @custom_model.partitioned_inference_api
    def predict(self, df: pd.DataFrame) -> pd.DataFrame:        
        from prophet import Prophet
        
        
        for column in df.columns:
            if column not in ['TRAFFIC','HOLIDAY_NAME','DATE']:
                df[column] = df[column].astype('int')
        
        holidays_df = df[['DATE', 'HOLIDAY_NAME']].rename(columns={'DATE': 'ds', 'HOLIDAY_NAME': 'holiday'})

        df = df.drop(columns = ['STORE_ID','HOLIDAY_NAME'])
        
        # # Filter the DataFrame
        train = df[(df['DATE'] >= train_start_date) & (df['DATE'] <= train_end_date)]
        test = df[(df['DATE'] >= test_start_date) & (df['DATE'] <= max_forecast_str)]
        
        # # Prepare data for Prophet
        prophet_df = train[['DATE', 'TRAFFIC']].rename(columns={'DATE': 'ds', 'TRAFFIC': 'y'})
        
        # List of extra features to add
        extra_features = [col for col in train.columns if col not in ['TRAFFIC']]
        extra_feature_df = train[extra_features].reset_index(drop=True)
        
        extra_feature_df.head()
        #Concatenate the extra features with the prophet_df
        prophet_df = pd.merge(prophet_df, extra_feature_df, left_on='ds', right_on='DATE', how='inner')
        
        # Initialize the Prophet model
        prophet_model = Prophet(holidays = holidays_df)
        
        # Add each extra feature as a regressor
        for feature in extra_features:
            prophet_model.add_regressor(feature)
        
        # Fit the model
        prophet_model.fit(prophet_df)
        
        # Create future DataFrame for Prophet
        future = prophet_model.make_future_dataframe(periods=len(test))
        
        # Add the extra features to the future DataFrame
        for feature in extra_features:
            future[feature] = pd.concat([train[feature], test[feature]]).reset_index(drop=True)
        
        # Get the Prophet predictions
        prophet_forecast = prophet_model.predict(future)
        
        # Extract Prophet's predictions for the test period
        prophet_test_forecast = prophet_forecast['yhat'].iloc[-len(test):].reset_index(drop=True)
        
        # Create DataFrame for Prophet results
        prophet_results = pd.DataFrame({
            'DATE': future['ds'].iloc[-len(test):].reset_index(drop=True),  # Use 'ds' for the future dates
            'Prophet_Prediction': prophet_test_forecast
        })
        
        # Ensure DATE is in datetime format
        prophet_results['DATE'] = pd.to_datetime(prophet_results['DATE'])

        return prophet_results

my_forecasting_model_prophet = ForecastingModel_prophet()

In [ ]:
model_name="PROPHET"
options = {
    "function_type": "TABLE_FUNCTION",
}

mv = reg.log_model(
    my_forecasting_model_prophet,
    model_name=model_name,
    version_name=get_next_version(reg,model_name),
    conda_dependencies=["pandas", "scikit-learn", "xgboost","prophet"],
    options=options,
    signatures={
        "predict": model_signature.ModelSignature(
            inputs=[
                model_signature.FeatureSpec(name="DATE", dtype=model_signature.DataType.TIMESTAMP_NTZ),
                model_signature.FeatureSpec(name="STORE_ID", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="WEEK_DAY_NBR", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="MTH_DAY_NBR", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="CALENDAR_MTH", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="CALENDAR_YEAR", dtype=model_signature.DataType.INT64),
                model_signature.FeatureSpec(name="HOLIDAY_NAME", dtype=model_signature.DataType.STRING),
                model_signature.FeatureSpec(name="TRAFFIC", dtype=model_signature.DataType.DOUBLE),
            ],
            outputs=[
                model_signature.FeatureSpec(name="DS", dtype=model_signature.DataType.TIMESTAMP_NTZ),
                model_signature.FeatureSpec(name="PREDICTION", dtype=model_signature.DataType.DOUBLE),
         ],
        )
    },
)

In [ ]:
results = mv.run(df, partition_column="STORE_ID").select(
    F.to_date("DS").as_('DATE'), "PREDICTION", "STORE_ID"
).cache_result()

final_PROPHET = results.join(
    df, ['Date','Store_ID']
).select(
    F.to_date(df.DATE).alias("DATE"),
    df.STORE_ID.alias("STORE_ID"),
    df.TRAFFIC,
    results.PREDICTION,
).with_column('MODEL',F.lit('PROPHET'))
final_PROPHET.show()

In [ ]:
mape_values = calculate_mape(
    final_PROPHET.filter(
        (F.col("DATE") >= test_start_date) & (F.col("DATE") <= test_end_date) & (F.col("STORE_ID") <= 10)
    )
)
mape_results = mape_values.collect()

# Create a nested dictionary with COUNTRY as the outer key and CHANNEL as the inner key
mape_dict = {}
for row in mape_results:
    store_id = row[0]
    mape_value = round(row[1], 1)
    if store_id not in mape_dict:
        mape_dict[store_id] = {}
    mape_dict[store_id] = f"{mape_value}%"

mape_json = json.dumps(mape_dict, indent=4)
print(mape_json)


In [ ]:
mv.set_metric(metric_name="MAPES", value=mape_json)

In [ ]:
final = final_PROPHET.union(final_XGB)

In [ ]:
from snowflake.snowpark.functions import col
# Create DataFrames for XGBOOST and PROPHET models
df_xgb = (
    final
    .filter(col("MODEL") == "XGBOOST")
    .select(
        "*", 
        (col("PREDICTION").alias("PRED_XGB"))
    )
).cache_result()

df_prophet = (
    final
    .filter(col("MODEL") == "PROPHET")
    .select(
        "*", 
        (col("PREDICTION").alias("PRED_PROPHET"))
    )
).cache_result()

# Join the two DataFrames
result_df = (
    df_xgb.join(
        df_prophet,
        (df_xgb["DATE"] == df_prophet["DATE"]) &
        (df_xgb["STORE_ID"] == df_prophet["STORE_ID"]),
        "inner"
    )
    .select(
        df_xgb["DATE"].alias('DATE'),
        df_xgb["STORE_ID"].alias('STORE_ID'),
        df_xgb["TRAFFIC"].alias('TRAFFIC'),
        df_xgb["PRED_XGB"],
        df_prophet["PRED_PROPHET"]
    )
)
result_df.write.mode("overwrite").save_as_table('final_store_predictions')
result_df.show()